In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import misfunciones as mf

In [2]:
sharks = pd.read_csv("data/attacks.csv",encoding = "ISO-8859-1")

In [3]:
#sharks

In [4]:
#limpiamos todas las columnas y filas que 
sharks.dropna(how="all", inplace=True)
sharks.dropna(how="all",axis = 1, inplace=True)

# Hipótesis
- hipotesis 1 los ataques de tiburones han icrmentado según han ido avanzando los años
- hipótesis 2 los ataquesde tiburones atacan más a embarcaciones que a nadadores
- hipóstesis 3 Florida es la capital mundial de los ataques de tiburones:
        https://www.lavanguardia.com/ocio/viajes/20210407/6631447/6-playas-mas-peligrosas-mundo.html
- hipótesis 4 ¿Es posible el ataque de tiburón en la costa española?
        https://www.mundo-geo.es/naturaleza/es-posible-ataque-tiburon-en-costa-espanola_238643_102.html

In [5]:
columnas = list(sharks.columns)
print(columnas)

['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href', 'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22', 'Unnamed: 23']


In [6]:
#columnas que no necesito para ninguna de mis hipótesis (primera pasasda)
columnas_no = ['Case Number', #no
'Investigator or Source', #no
 'pdf', #no
 'href formula', #no
 'href', #no
 'Case Number.1', #no
 'Case Number.2', #no
 'original order', #no
 'Unnamed: 22',#no
 'Unnamed: 23'] #no

sharks.drop(columnas_no, axis=1, inplace=True)

In [7]:
#ahora que hemos borrado columnas vemos si quedan filas que son nan por completo y borrranos
sharks.dropna(how="all", inplace=True)

In [8]:
columnas = sharks.columns
columnas

Index(['Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity',
       'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species '],
      dtype='object')

In [9]:
#areas = pd.DataFrame(sharks.Area.value_counts())
#areas

### Limpieza de datos nulos

In [10]:
sharks.isna().sum() #miramos cuantos datos nulos hay en cda columna.

Date              0
Year              2
Type              4
Country          50
Area            455
Location        540
Activity        544
Name            210
Sex             565
Age            2831
Injury           28
Fatal (Y/N)     539
Time           3354
Species        2838
dtype: int64

In [11]:
sharks.Date.sample(20) #limpiar fechas por día, mes y año.
#quitar reported, circa
sharks.Date.value_counts()

1957                          11
1942                           9
1956                           8
1958                           7
1950                           7
                              ..
01-Dec-1994                    1
Reported      10-Dec-1994      1
11-Dec-1994                    1
13-Dec-1994                    1
1845-1853                      1
Name: Date, Length: 5433, dtype: int64

In [12]:
sharks.Type.unique() #solo hay 4 nulos

array(['Boating', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable',
       'Sea Disaster', nan, 'Boat', 'Boatomg'], dtype=object)

In [13]:
sharks[sharks.Type == "Invalid"].sample(5) #miramos a ver si los datos nulos pueden sustitirse por "Ivalid"

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species
5933,Reported 24-Apr-1866,1866.0,Invalid,AUSTRALIA,Western Australia,Off De Grey river,Fell overboard,Mr. Groves,M,NaN,Thought to have been taken by a shark. Body wa...,NaN,NaN,Shark involvement prior to death unconfirmed
1805,20-Dec-2002,2002.0,Invalid,AUSTRALIA,Queensland,Golden Beach,Swimming,Bayne Doyle,M,11,Foot lacerated,NaN,NaN,"Shark involvement not confirmed, injury may be..."
3493,Reported 21-Dec-1967,1967.0,Invalid,PAPUA NEW GUINEA,Central Province,Port Moresby,Spearfishing,Bob Valentine,M,32,No injury,NaN,NaN,Questionable incident
3491,1968,1968.0,Invalid,USA,Florida,"Jensen Beach, Martin County",Surfing,NaN,NaN,NaN,NaN,NaN,17h00,Questionable incident
402,26-Jun-2015,2015.0,Invalid,USA,Florida,"Jacksonville Beach, Duval County",Swimming,female,F,NaN,Minor lacerations to leg,NaN,19h30,Shark involvement not confirmed


In [14]:
sharks.Type.fillna("Invalid",inplace = True)

In [15]:
sharks["Fatal (Y/N)"].unique()

array(['N', 'Y', nan, 'M', 'UNKNOWN', '2017', ' N', 'N ', 'y'],
      dtype=object)

In [16]:
sharks["Fatal (Y/N)"].fillna("UNKNOWN",inplace = True)
sharks["Fatal (Y/N)"].unique() #hay que luego limpiar los datos para que no haya espacios y queden solo N,Y,UNKOWN

array(['N', 'Y', 'UNKNOWN', 'M', '2017', ' N', 'N ', 'y'], dtype=object)

In [17]:
sharks["Injury"].unique()

array(['No injury to occupant, outrigger canoe and paddle damaged',
       'Minor injury to left thigh',
       'Injury to left lower leg from surfboard skeg', ...,
       'FATAL, leg stripped of flesh  ',
       'FATAL, knocked overboard by tail of shark & carried off by shark ',
       'FATAL. "Shark bit him in half, carrying away the lower extremities" '],
      dtype=object)

In [18]:
sharks["Injury"].fillna("UNKNOWN",inplace = True) #rellenamos también injury con UNKOWN para los nan 
#quizas quite esta columna más tarde

In [19]:
sharks["Name"].unique()
sharks["Name"].fillna("John/Jane Doe",inplace = True)
#puede que la quitemos, porque con saber el sexo de la persona nos sobra

In [20]:
#sharks["Species"].replace(".*[wW](hite|HITE).*","White",inplace = True)

In [21]:
#cambiamos year a enteros
sharks["Year"].fillna(0.0,inplace = True)
sharks["Year"] = sharks["Year"].apply(mf.entero)

In [22]:
sharks["Sex "].unique()
sharks["Sex "].fillna("X", inplace = True)
#luego cambiar nombre columna y cambiar datos que no sean M, F, o X para desconocido.

In [23]:
actividades = sharks["Activity"].value_counts() #con esto se hace un objeto que nos puede servir de diccionario

In [24]:
sharks["Country"].unique()
sharks["Country"].fillna("UNKNOWN", inplace = True)

In [25]:
sharks["Age"].fillna(0, inplace = True)

In [26]:
sharks["Area"].fillna("UNKNOWN", inplace = True)
sharks["Location"].fillna("UNKNOWN", inplace = True)
sharks["Activity"].fillna("UNKNOWN", inplace = True)
sharks["Species "].fillna("UNKNOWN", inplace = True) #cambiar nombre de la columna a "Species""
sharks["Time"].fillna("UNKNOWN", inplace = True)

In [27]:
#cambio nombre de algunas columnas que tienen espacios o caracteres raros en el nombre
list(columnas) #"Sex " "Species " Fatal (Y/N)
cols = ["Sex ","Species ","Fatal (Y/N)"]
new_names = {"Sex " : "Sex",
                 "Species " : "Species",
                 "Fatal (Y/N)" : "Fatal"}
sharks.rename(columns = new_names, inplace=True)
sharks.columns

Index(['Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity',
       'Name', 'Sex', 'Age', 'Injury', 'Fatal', 'Time', 'Species'],
      dtype='object')

In [28]:
sharks["Sex"].unique() #'M ', 'lli', 'N', '.' -- X = desconocido
sharks["Sex"].replace('M ',"M",inplace=True)
sharks["Sex"].replace('lli',"X",inplace=True)
sharks["Sex"].replace('.',"X",inplace=True)
sharks["Sex"].replace('N',"X",inplace=True)

In [29]:
sharks["Sex"].unique()

array(['F', 'M', 'X'], dtype=object)

In [30]:
sharks[(sharks.Country == "UNKNOWN" )&(sharks.Area == "UNKNOWN" )&(sharks.Location == "UNKNOWN" )] #29 líneas ¿las borro?

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species
62,Reported 13-Nov-2017,2017,Unprovoked,UNKNOWN,UNKNOWN,UNKNOWN,Surfing,Timur Yunusov,M,24,Puncture wounds to feet,N,UNKNOWN,UNKNOWN
525,Aug-2014,2014,Invalid,UNKNOWN,UNKNOWN,UNKNOWN,Sea disaster,Cuban refugees,M,0,Shark involvement prior to death not confirmed,UNKNOWN,UNKNOWN,Shark involvement not confirmed
3378,Nov-1970,1970,Unprovoked,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,Heinz Plotsky,M,0,Extensive injuries,N,UNKNOWN,UNKNOWN
3388,5-Jul-1970,1970,Unprovoked,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,male,M,0,Finger or toe severed,N,Night,Mako shark
3395,Apr-1970,1970,Provoked,UNKNOWN,UNKNOWN,UNKNOWN,Freediving,Lionel Jarvis,M,0,Arm abraded & lacerated. Recorded as PROVOKED ...,N,UNKNOWN,Wobbegong shark
3399,5-Feb-1970,1970,Unprovoked,UNKNOWN,UNKNOWN,UNKNOWN,Wading,Sally Anne Irvine,F,8,Lacerations to lower leg,N,UNKNOWN,Carpet shark
3425,Aug-1969,1969,Unprovoked,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,Rodney Hughes,M,25,Am lacerated,N,UNKNOWN,UNKNOWN
3661,27-Sep-1964,1964,Invalid,UNKNOWN,UNKNOWN,UNKNOWN,Spearfishing,Giancarlo Griffon,M,24,"Disappeared, probable drowning but sharks in a...",UNKNOWN,11h00,Shark involvement prior to death was not confi...
4726,1942,1942,Sea Disaster,UNKNOWN,UNKNOWN,UNKNOWN,Jumped overboard from torpedoed Panamanian fre...,male,M,0,FATAL,Y,UNKNOWN,UNKNOWN
4729,1942,1942,Boating,UNKNOWN,UNKNOWN,UNKNOWN,"Days before the surrender of Singapore, the 3 ...","Bombardier J. Hall, Private Green of the Sherw...",M,0,No injury to occupants. Sharks continually fol...,N,UNKNOWN,UNKNOWN


In [31]:
list(sharks["Date"].unique())
#eliminar "Reported "
#[rR](eported|EPORTED)\s\d{2}.[A-Z][a-z][a-z].\d{4}"
sharks["Date"] = sharks["Date"].str.extract(r'(\d{2}.[A-Z][a-z][a-z].\d{4})')


In [37]:
sharks["Date"].fillna("00-000-0000", inplace = True)

In [39]:
sharks["Month"] = sharks["Date"].str.extract(r'([A-Z][a-z]{2})')

In [41]:
sharks["Month"].fillna("000",inplace = True)

In [57]:

sharks[(sharks.Year < 1543)&(sharks.Year >0)]

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species,Month
6174,00-000-0000,500,Unprovoked,MEXICO,UNKNOWN,UNKNOWN,UNKNOWN,male,M,0,Foot severed,N,UNKNOWN,UNKNOWN,000
6175,00-000-0000,77,Unprovoked,UNKNOWN,Ionian Sea,UNKNOWN,Sponge diving,males,M,0,FATAL,Y,UNKNOWN,UNKNOWN,000
6176,00-000-0000,5,Unprovoked,AUSTRALIA,New South Wales,Bondi,UNKNOWN,male,M,0,Aboriginal rock carving depicts man being atta...,N,UNKNOWN,UNKNOWN,000


In [50]:
sharks.Year.unique()

array([2018, 2017,    0, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009,
       2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998,
       1997, 1996, 1995, 1984, 1994, 1993, 1992, 1991, 1990, 1989, 1969,
       1988, 1987, 1986, 1985, 1983, 1982, 1981, 1980, 1979, 1978, 1977,
       1976, 1975, 1974, 1973, 1972, 1971, 1970, 1968, 1967, 1966, 1965,
       1964, 1963, 1962, 1961, 1960, 1959, 1958, 1957, 1956, 1955, 1954,
       1953, 1952, 1951, 1950, 1949, 1948, 1848, 1947, 1946, 1945, 1944,
       1943, 1942, 1941, 1940, 1939, 1938, 1937, 1936, 1935, 1934, 1933,
       1932, 1931, 1930, 1929, 1928, 1927, 1926, 1925, 1924, 1923, 1922,
       1921, 1920, 1919, 1918, 1917, 1916, 1915, 1914, 1913, 1912, 1911,
       1910, 1909, 1908, 1907, 1906, 1905, 1904, 1903, 1902, 1901, 1900,
       1899, 1898, 1897, 1896, 1895, 1894, 1893, 1892, 1891, 1890, 1889,
       1888, 1887, 1886, 1885, 1884, 1883, 1882, 1881, 1880, 1879, 1878,
       1877, 1876, 1875, 1874, 1873, 1872, 1871, 18